<a href="https://colab.research.google.com/github/deepakupman/dogs_and_cats/blob/master/dog_cat_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount("/content/gdrive")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
import torch
from torchvision import datasets, transforms
gpu_available = torch.cuda.is_available()
if gpu_available:
    print("Training on GPU")

Training on GPU


In [0]:
data_dir = "/content/gdrive/My Drive/dataset/cat_dog_ds"
batch_size = 32
transform = transforms.Compose([transforms.RandomRotation(30),
                                transforms.Resize(256),
                                transforms.RandomCrop(224),
                                transforms.RandomHorizontalFlip(),
                                transforms.RandomVerticalFlip(),
                                transforms.ToTensor(),
                                transforms.Normalize([0.5,0.5,0.5],[0.5,0.5,0.5])])

train_data = datasets.ImageFolder(data_dir + "/training_set/", transform=transform)
test_data = datasets.ImageFolder(data_dir + "/test_set/", transform=transform)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=True)

In [0]:
img, lbl = next(iter(train_loader))

In [5]:
img.size()

torch.Size([32, 3, 224, 224])

In [0]:
from torch import nn
from torch import optim
import torch.nn.functional as F

In [0]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        
        # convolution layers
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.conv3 = nn.Conv2d(32, 64, 3, padding=1)
        
        # pooling layer
        self.pool = nn.MaxPool2d(2,2)
        
        # fully connected layers
        self.fc1 = nn.Linear(64*28*28, 512)
        
        # dropout layer
        self.dropout = nn.Dropout(0.3)
        
        self.fc2 = nn.Linear(512, 2)
        
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        # flatten the layer
        x = x.view(-1, 64 * 28 * 28)
        
        x = self.dropout(F.relu(self.fc1(x)))
        
        return x

In [8]:
model = Model()
# loss
criterion = nn.CrossEntropyLoss()
# optimizer
opt = optim.Adam(model.parameters(), lr=0.01)
print(model)
if gpu_available:
    model.cuda()

Model(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=50176, out_features=512, bias=True)
  (dropout): Dropout(p=0.3)
  (fc2): Linear(in_features=512, out_features=2, bias=True)
)


In [9]:
# epochs
for e in range(5):
  
  # keep track of train and validation loss
  train_loss = 0.0
  valid_loss = 0.0
  
  # enable training mode
  model.train()
  
  # train the model
  for image, label in train_loader:
    # check for GPU
    if gpu_available:
        image, label = image.cuda(), label.cuda()
    # make gradient zero
    opt.zero_grad()
    # get output
    out = model(image)
    # calc. loss
    loss = criterion(out, label)
    # backward propagation
    loss.backward()
    # calculate gradient descent
    opt.step()
    # add train loss
    train_loss += loss.item() * image.size(0)
      
  # valid the model
#   for img, lbl in valid_loader:
#     # check for gpu
#     if gpu_available:
#       img, lbl = img.cuda(), lbl.cuda()
#     # get output
#     output = model(img)
#     # calculate loss
#     loss = criterion(output, lbl)
#     # add valid loss
#     valid_loss += loss.item() * img.size(0)

  # calculate average loss
  train_loss = train_loss / len(train_loader.dataset)
#   valid_loss = valid_loss / len(valid_loader.dataset)
  
  print("Epoch: {}".format(e+1),
        "Loss: {}".format(train_loss))
#         "Valid Loss: {}".format(valid_loss))

Epoch: 1 Loss: 1.9230275077819825
Epoch: 2 Loss: 1.9260826425552369
Epoch: 3 Loss: 1.8574613652229308
Epoch: 4 Loss: 1.901129346847534
Epoch: 5 Loss: 1.7981969704627991
